In [91]:
# ** 1. Test reattach_fragments **

In [92]:
import itertools

import rmgpy

from rmgpy.data.rmg import RMGDatabase
from rmgpy.rmg.react import *
from rmgpy.reaction import Reaction
from rmgpy.molecule.molecule import Molecule
from rmgpy.molecule.resonance import *
from rmgpy.species import Species
from rmgpy.data.kinetics.common import filter_reactions

from rmgpy.data.rmg import getDB
from rmgpy.scoop_framework.util import map_
from copy import deepcopy
from IPython.display import display

In [93]:
import numpy as np

In [100]:
import os
import shutil
import unittest
import afm.utils_1_sided
import afm.simulator_2mo
import afm.molecule

In [101]:
        chemkin_path = os.path.join('/home/yentingw/AutoFragmentModeling/test/',
                                    'data', 
                                    '2mo_simulator_data',
                                    'chem.inp')

        dictionary_path = os.path.join('/home/yentingw/AutoFragmentModeling/test/',
                                    'data', 
                                    '2mo_simulator_data',
                                    'species_dictionary.txt')

        fragment_smiles_path = os.path.join('/home/yentingw/AutoFragmentModeling/test/',
                                    'data', 
                                    '2mo_simulator_data',
                                    'fragment_smiles.txt')

        temperature = 673.15 # unit: K
        pressure = 350*3.75 # unit: bar
       
        odes = afm.simulator_2mo.OdeSimulator(chemkin_path,
                                              dictionary_path,
                                              fragment_smiles_path,
                                              temperature,
                                              pressure,)
                                              

In [102]:
odes

In [103]:
        initial_mol_fraction = {
                                "ArCC(C)R":1.0,
                                "RCCCCR":1.0,
                                "RCC":1.0
                                }

        termination_time = 3600*14 # unit: sec
        all_data = odes.simulate(initial_mol_fraction, termination_time)

In [104]:
        _, dataList, _ = all_data[0]
        TData = dataList[0]
        PData = dataList[1]
        VData = dataList[2]
        total_moles = PData.data*VData.data/8.314/TData.data
        moles_dict = {}
        for data in dataList[3:]:
            spe_label = data.label
            if '*' in spe_label:
                continue
            r_count = spe_label.count('R')
           
            label_count = r_count

            if label_count == 0:
                continue
            if abs(data.data[-1]*total_moles[-1]) <= 1e-6:
                continue
            moles_dict[spe_label] = max(data.data[-1]*total_moles[-1],0)

In [105]:
TData.data

array([ 673.15,  673.15,  673.15, ...,  673.15,  673.15,  673.15])

In [106]:
moles_dict

{'ArCC(C)R': 6433.2400619713653,
 'ArC__CR': 6.1267021129075161,
 'RC': 616.29133264348195,
 'RC(C)C': 1.4590445968795553e-06,
 'RC(C)__C': 6.0987455757229654,
 'RCC': 7936.3532428969165,
 'RCCC': 0.016806676674460327,
 'RCCCCR': 7200.9062906492891,
 'RCC__C': 616.3021904477356,
 'RCC__CCR': 0.058619319624187038,
 'RC__C': 1027.0737953680355}

## Try simple example of fragment sequence

In [49]:
moles_dict = {}
spe_label = ['ArCC(C)R', 'RCC', 'RCCCCR', 'RCC__C', 'ArC__CR', 'RCC__CCR']
values = [6, 2, 3, 6, 4, 3]
i=0

for spe_name in spe_label:
    moles_dict[spe_name] = values[i]
    i += 1 

In [50]:
moles_dict

{'ArCC(C)R': 6,
 'ArC__CR': 4,
 'RCC': 2,
 'RCCCCR': 3,
 'RCC__C': 6,
 'RCC__CCR': 3}

In [107]:
r_moles, _, rr_list = afm.simulator_2mo.categorize_fragments(moles_dict)

In [108]:
r_moles

[('RCC__CCR', 0.11723863924837408),
 ('RCCCCR', 14401.812581298578),
 ('ArC__CR', 6.1267021129075161),
 ('RCC__C', 616.3021904477356),
 ('ArCC(C)R', 6433.2400619713653),
 ('RCCC', 0.016806676674460327),
 ('RC(C)C', 1.4590445968795553e-06),
 ('RC', 616.29133264348195),
 ('RC(C)__C', 6.0987455757229654),
 ('RC__C', 1027.0737953680355),
 ('RCC', 7936.3532428969165)]

In [109]:
rr_list

['RCC__CCR', 'RCCCCR']

In [110]:
# test reattchment
grind_size = 1
shuffle_seed = 0
grinded_r_moles = afm.utils_1_sided.grind(r_moles, grind_size)

In [112]:
grinded_r_moles

[('RCC__CCR', 0.11723863924837408),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('R

In [113]:
r_moles_shuffle = afm.utils_1_sided.shuffle(grinded_r_moles, shuffle_seed)

In [114]:
r_moles_shuffle

[('RCCCCR', 1),
 ('ArCC(C)R', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('ArCC(C)R', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('ArCC(C)R', 1),
 ('RCCCCR', 1),
 ('ArCC(C)R', 1),
 ('ArCC(C)R', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RC__C', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('RC__C', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('ArCC(C)R', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCC__C', 1),
 ('ArCC(C)R', 1),
 ('ArCC(C)R', 1),
 ('RCC', 1),
 ('ArCC(C)R', 1),
 ('RC', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCC', 1),
 ('ArCC(C)R', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('ArCC(C)R', 1),
 ('RCC', 1),
 ('ArCC(C)R', 1),
 ('RCC', 1),
 ('RCC', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1),
 ('RCCCCR', 1

In [58]:
half_length = len(r_moles_shuffle)/2
half_r_moles_shuffle_1 = r_moles_shuffle[0:half_length]
half_r_moles_shuffle_2 = r_moles_shuffle[half_length:len(r_moles_shuffle)]

In [60]:
matches0 = afm.utils_1_sided.match_concentrations_with_same_sums(half_r_moles_shuffle_1,
																 half_r_moles_shuffle_2,
													   			 diff_tol=1e-3)
matches0

[(('RCC__CCR', 'ArCC(C)R'), 1),
 (('RCC__C', 'RCC__CCR'), 1),
 (('ArC__CR', 'RCC__C'), 1),
 (('RCCCCR', 'RCCCCR'), 1),
 (('RCC__C', 'RCC__CCR'), 1),
 (('RCC__C', 'ArCC(C)R'), 1),
 (('RCC__C', 'ArCC(C)R'), 1),
 (('RCCCCR', 'RCCCCR'), 1),
 (('ArCC(C)R', 'RCCCCR'), 1),
 (('RCC', 'RCC__C'), 1),
 (('RCC__CCR', 'RCC__CCR'), 1),
 (('ArCC(C)R', 'RCC__CCR'), 1),
 (('RCCCCR', 'ArCC(C)R'), 1),
 (('ArC__CR', 'ArC__CR'), 1),
 (('RCC', 'ArC__CR'), 1)]

In [61]:
matches1, new_r_l_moles = afm.utils_1_sided.matches_resolve(matches0, rr_list)
matches1

[(('ArCC(C)R', 'RCC__CCR', 'ArCC(C)R'), 0.5),
 (('RCC__C', 'RCC__CCR', 'RCC__C'), 0.5),
 (('ArC__CR', 'RCC__C'), 1),
 (('RCC__C', 'RCC__CCR', 'RCC__C'), 0.5),
 (('RCC__C', 'ArCC(C)R'), 1),
 (('RCC__C', 'ArCC(C)R'), 1),
 (('ArCC(C)R', 'RCCCCR', 'ArCC(C)R'), 0.5),
 (('RCC', 'RCC__C'), 1),
 (('ArCC(C)R', 'RCC__CCR', 'ArCC(C)R'), 0.5),
 (('ArCC(C)R', 'RCCCCR', 'ArCC(C)R'), 0.5),
 (('ArC__CR', 'ArC__CR'), 1),
 (('RCC', 'ArC__CR'), 1)]

In [79]:
r_r_moles=[]
r_r_moles.extend(new_r_l_moles)
matches = afm.utils_1_sided.match_concentrations_with_different_sums(matches1, r_r_moles)

In [80]:
matches

[((('ArCC(C)R', 'RCC__CCR', 'ArCC(C)R'), ('RCCCCR', 'RCCCCR')), 0.5),
 ((('RCC__C', 'RCC__CCR', 'RCC__C'), ('RCCCCR', 'RCCCCR')), 0.5),
 ((('ArC__CR', 'RCC__C'), ('RCC__CCR', 'RCC__CCR')), 0.5),
 (('ArC__CR', 'RCC__C'), 0.5),
 (('RCC__C', 'RCC__CCR', 'RCC__C'), 0.5),
 (('RCC__C', 'ArCC(C)R'), 1),
 (('RCC__C', 'ArCC(C)R'), 1),
 (('ArCC(C)R', 'RCCCCR', 'ArCC(C)R'), 0.5),
 (('RCC', 'RCC__C'), 1),
 (('ArCC(C)R', 'RCC__CCR', 'ArCC(C)R'), 0.5),
 (('ArCC(C)R', 'RCCCCR', 'ArCC(C)R'), 0.5),
 (('ArC__CR', 'ArC__CR'), 1),
 (('RCC', 'ArC__CR'), 1)]

In [81]:
        moles_dict_after_match = {}
        for match in matches:
            combo, val = match
            frags = afm.utils_1_sided.flatten(combo)
            for frag in frags:
                if frag not in moles_dict_after_match:
                    moles_dict_after_match[frag] = val
                else:
                    moles_dict_after_match[frag] += val

        for frag, val in moles_dict.iteritems():
            val_after_match = moles_dict_after_match[frag]
            diff_pct = abs(val - val_after_match)/moles_dict[frag]

In [90]:
unittest.TestCase().assertAlmostEqual(diff_pct, 0.0, 6)

ValueError: no such test method in <class 'unittest.case.TestCase'>: runTest

In [84]:
val

6